In [ ]:
import json
import os
import pandas as pd
from time import sleep
from threading import Thread
from os.path import join, exists
from traceback import print_exc
from random import random
from datetime import datetime, timezone, timedelta, date
from api.dwx_client import dwx_client
import numpy as np
import pandas_ta as ta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from backtesting import Strategy, Backtest
from apscheduler.schedulers.blocking import BlockingScheduler
import sys
from threading import Event
import pytz

class getdf():
    def __init__(self, MT4_directory_path, 
                 sleep_delay=0.005,             # 5 ms for time.sleep()
                 max_retry_command_seconds=10,  # retry to send the commend for 10 seconds if not successful. 
                 verbose=True, ):
        self.data_event = Event()
        self.dfstream = None

        self.dwx = dwx_client(self, MT4_directory_path, sleep_delay, 
                              max_retry_command_seconds, verbose=verbose)
        
        self.dwx.start()
        end = datetime.now(timezone.utc) + timedelta(hours=4)
        start = end - timedelta(days=300)  # últimos 300 días
        self.dwx.get_historic_data('EURUSD', 'M5', start.timestamp(), end.timestamp())    

    def on_historic_data(self, symbol, time_frame, data):
        data_list = []

        for date_str, values in data.items():
            date = datetime.strptime(date_str, "%Y.%m.%d %H:%M")
            data_dict = {
                'Date': date,
                'Open': values['open'],
                'High': values['high'],
                'Low': values['low'],
                'Close': values['close'],
                'Tick_volume': values['tick_volume']
            }
            data_list.append(data_dict)
        
        self.dfstream = pd.DataFrame(data_list)
        print("Historic DataFrame (300 days):")
        print(self.dfstream)
        
        self.data_event.set() 

    def on_message(self, message):
        if message['type'] == 'ERROR':
            print(message['type'], '|', message['error_type'], '|', message['description'])
        elif message['type'] == 'INFO':
            print(message['type'], '|', message['message'])
            
    def get_data_frame(self):
        self.data_event.wait()  # Wait until data is ready
        return self.dfstream

# Example usage
MT4_files_dir = r'C:\Users\guill\AppData\Roaming\MetaQuotes\Terminal\6C3C6A11D1C3791DD4DBF45421BF8028\MQL5\Files'
if not exists(MT4_files_dir):
    print('ERROR: metatrader_dir_path does not exist!')
    sys.exit()

data_processor1 = getdf(MT4_files_dir)

# Wait for data to be processed and retrieve the DataFrame
df_300_days = data_processor1.get_data_frame()

class GetDF1():
    def __init__(self, MT4_directory_path, 
                 sleep_delay=0.005,             # 5 ms for time.sleep()
                 max_retry_command_seconds=10,  # retry to send the commend for 10 seconds if not successful. 
                 verbose=True, ):
        self.data_event1 = Event()
        self.dfstream1 = None

        self.dwx1 = dwx_client(self, MT4_directory_path, sleep_delay, 
                              max_retry_command_seconds, verbose=verbose)
        
        self.dwx1.start()
        tz1 = pytz.timezone('Etc/GMT-4')
        end1 = datetime.now(tz1)
        start1 = end1 - timedelta(minutes=60) 
        
        self.dwx1.get_historic_data('EURUSD', 'M5', start1.timestamp(), end1.timestamp())    
  
    def on_historic_data(self, symbol, time_frame, data1):
        data_list1 = []

        for date_str, values in data1.items():
            date = datetime.strptime(date_str, "%Y.%m.%d %H:%M")
            data_dict1 = {
                'Date': date,
                'Open': values['open'],
                'High': values['high'],
                'Low': values['low'],
                'Close': values['close'],
                'Tick_volume': values['tick_volume']
            }
            data_list1.append(data_dict1)
        
        self.dfstream1 = pd.DataFrame(data_list1)
        print("The LAST ROW:")
        print(self.dfstream1)
        
        self.data_event1.set() 

    def on_message(self, message):
        if message['type'] == 'ERROR':
            print(message['type'], '|', message['error_type'], '|', message['description'])
        elif message['type'] == 'INFO':
            print(message['type'], '|', message['message'])
            
    def get_data_frame1(self):
        self.data_event1.wait()  # Wait until data is ready
        return self.dfstream1

# Example usage
MT4_files_dir1 = r'C:\Users\guill\AppData\Roaming\MetaQuotes\Terminal\6C3C6A11D1C3791DD4DBF45421BF8028\MQL5\Files'
if not exists(MT4_files_dir1):
    print('ERROR: metatrader_dir_path does not exist!')
    sys.exit()

last_row_processor = GetDF1(MT4_files_dir1)

# Wait for data to be processed and retrieve the DataFrame
df1 = last_row_processor.get_data_frame1()






class tick_processor:

    def __init__(self, data_processor1, last_row_processor, MT4_directory_path, sleep_delay=0.005, max_retry_command_seconds=10, verbose=True):
     
        try:
            self.open_test_trades = True
            self.last_open_time = datetime.now(timezone.utc)
            self.last_modification_time = datetime.now(timezone.utc)
            # Assuming dwx_client is defined elsewhere and correctly imported
            self.dwx2 = dwx_client(self, MT4_directory_path, sleep_delay, max_retry_command_seconds, verbose=verbose)
            sleep(1)
            self.dwx2.start()
            print("Account info:", self.dwx2.account_info)
            
             # Initialize dfstream here
            self.slatrcoef = 0
            self.TPSLRatio_coef = 0
            self.initsize = 0.99
            self.mysize = self.initsize
            self.original_df = data_processor1.get_data_frame()
            self.last_row = last_row_processor.get_data_frame1()
            
              # últimos 100 días
            
        except Exception as e:
            print("Error initializing tick_processor:", e)
            print_exc()
            
    


    def transformdata(self,dfstream):    
       
        try:

            dfstream1 = dfstream.copy()
            df = dfstream1
            
            print(df)
            print('FASE0')
            print(df)
            df = df[df.High != df.Low]
            df['Date'] = pd.to_datetime(df['Date'])
            df.set_index('Date', inplace=True)
            df["VWAP"] = ta.vwap(df.High, df.Low, df.Close, df.Tick_volume)
            df['RSI'] = ta.rsi(df.Close, length=16)
            my_bbands = ta.bbands(df.Close, length=14, std=2.0)
            df = df.join(my_bbands)
            df.dropna(inplace=True)
            df['ATR'] = ta.atr(df.High, df.Low, df.Close, length=7)
            df = df.reset_index()
            
            VWAPsignal = [0] * len(df)
            backcandles = 15

            for row in range(backcandles, len(df)):
                upt = 1
                dnt = 1
                for i in range(row-backcandles, row+1):
                    if max(df.Open.iloc[i], df.Close.iloc[i]) >= df.VWAP.iloc[i]:
                        dnt = 0
                    if min(df.Open.iloc[i], df.Close.iloc[i]) <= df.VWAP.iloc[i]:
                        upt = 0
                if upt == 1 and dnt == 1:
                    VWAPsignal[row] = 3
                elif upt == 1:
                    VWAPsignal[row] = 2
                elif dnt == 1:
                    VWAPsignal[row] = 1

            df['VWAPSignal'] = VWAPsignal
            
            return df
        except Exception as e:
            print("Error in trasnformdata:", e)
            print_exc()

        
       

    def total_signal(self, dfstream):
       if dfstream is None:
          print("El DataFrame es None")
          return None

       def calculate_signal(l):
          if dfstream.VWAPSignal.iloc[l] == 2 and dfstream.Close.iloc[l] <= dfstream['BBL_14_2.0'].iloc[l] and dfstream.RSI.iloc[l] < 45:
              return 2
          if dfstream.VWAPSignal.iloc[l] == 1 and dfstream.Close.iloc[l] >= dfstream['BBU_14_2.0'].iloc[l] and dfstream.RSI.iloc[l] > 55:
            return 1
          return 0

       TotSignal = [0] * len(dfstream)
       backcandles = 15
       for row in range(backcandles, len(dfstream)):
           TotSignal[row] = calculate_signal(row)
       dfstream['TotalSignal'] = TotSignal

       signal_count = dfstream[dfstream.TotalSignal != 0].count()
       print(f"FASE2: {signal_count}")
       return dfstream
   

    def on_tick(self, symbol):
            

            print("El valor del dataframe original")


            print(self.original_df)
            print("El valor del dataframe last row")
            print(self.last_row)





            print("Processing new tick data...")
            print(self.original_df)
            last_row = self.last_row
            self.updated_df = pd.concat([self.original_df, last_row], ignore_index=True)

            print("Luego de añadir la ultima fila")    
            print(self.updated_df)
            self.updated_df = self.transformdata(self.updated_df)
            print("Luego de transformar data")   
            print(self.updated_df)
            signal_df = self.total_signal(self.updated_df)
            print("Luego de agregar total signal")  
            print(signal_df)
            if signal_df is not None:
                self.updated_df = signal_df
             
            print("FASE3")
            print(self.updated_df)
            self.dfstream = self.updated_df
            print("FASE4")
            print(self.dfstream)
            #now = datetime.now(timezone.utc)
            #print('on_tick:', now, symbol, bid, ask)
            slatr = 1.0 * self.dfstream['ATR'].iloc[-1] * self.slatrcoef
            TPSLRatio = self.TPSLRatio_coef
            try:
                slatr = 1.0 * self.dfstream['ATR'].iloc[-1] * self.slatrcoef
                TPSLRatio = self.TPSLRatio_coef
                print(f"slatr: {slatr}, TPSLRatio: {TPSLRatio}")

                # Close open trades under certain conditions
                for ticket in self.dwx2.open_orders.keys():
                    trade = self.dwx2.open_orders[ticket]
                    print(f"Checking trade {ticket}: {trade}")
                    if trade['type'] == 'buy' and self.dfstream['RSI'].iloc[-1] >= 90:
                       print(f"Closing buy trade {ticket} due to RSI >= 90")
                       self.dwx2.close_order(ticket)
                    elif trade['type'] == 'sell' and self.dfstream['RSI'].iloc[-1] <= 10:
                        print(f"Closing sell trade {ticket} due to RSI <= 10")
                        self.dwx2.close_order(ticket)

                # Open new trades based on the signal
                signal = self.dfstream['TotalSignal'].iloc[-1]
                print(f"Signal: {signal}")
                if signal == 2:  # Signal to buy
                   entry_price = self.dfstream.Close.iloc[-1]
                   sl1 = entry_price - slatr
                   tp1 = entry_price + slatr * TPSLRatio
                   print(f"Opening buy order: entry_price={entry_price}, sl1={sl1}, tp1={tp1}")
                   self.dwx2.open_order(symbol=symbol, order_type='buy', price=ask, sl=sl1, tp=tp1, lots=self.mysize)
                elif signal == 1:  # Signal to sell
                   entry_price = self.dfstream.Close.iloc[-1]
                   sl1 = entry_price + slatr
                   tp1 = entry_price - slatr * TPSLRatio
                   print(f"Opening sell order: entry_price={entry_price}, sl1={sl1}, tp1={tp1}")
                   self.dwx2.open_order(symbol=symbol, order_type='sell', price=bid, sl=sl1, tp=tp1, lots=self.mysize)
            except Exception as e:
                 print("Error in on_tick:", e)


            
    def on_message(self, message):

        if message['type'] == 'ERROR':
            print(message['type'], '|', message['error_type'], '|', message['description'])
        elif message['type'] == 'INFO':
            print(message['type'], '|', message['message'])
   
    def on_order_event(self):

        print(f'on_order_event. open_orders: {len(self.dwx.open_orders)} open orders')
    def get_dwx(self):
        return self.dwx
MT4_files_dir = r'C:\Users\guill\AppData\Roaming\MetaQuotes\Terminal\6C3C6A11D1C3791DD4DBF45421BF8028\MQL5\Files'
if not exists(MT4_files_dir):
        print('ERROR: metatrader_dir_path does not exist!')
        sys.exit()

processor = tick_processor(data_processor1,last_row_processor, MT4_files_dir)

# Wait for data to be processed and retrieve the DataFrame
scheduler = BlockingScheduler()
scheduler.add_job(lambda: processor.on_tick('EURUSD'), 'cron', day_of_week='mon-fri', hour='07-23', minute='1, 6, 11, 16, 21, 26, 31, 36, 41, 46, 51, 56', timezone='Asia/Beirut', misfire_grace_time=15)
scheduler.start()